## Amazon Bedrock Knowledge Bases가 제공하는 고급 청킹 전략

이 노트북에서는 Amazon Bedrock Knowledge Bases가 지원하는 다음 청킹 옵션에 대한 샘플 코드를 제공하기 위해 3개의 지식 베이스를 생성할 것입니다:
1. 고정 청킹(Fixed chunking)
2. 의미론적 청킹(Semantic chunking)
3. 계층적 청킹(Hierarchical chunking)
4. Lambda 함수를 사용한 사용자 정의 청킹(Custom chunking)
청킹은 임베딩하기 전에 텍스트를 더 작은 세그먼트로 분할합니다. 데이터 소스를 생성한 후에는 청킹 전략을 수정할 수 없습니다.
현재 Amazon Bedrock Knowledge Bases는 청킹 없음, 고정 크기 청킹, 기본 청킹과 같은 몇 가지 내장 청킹 옵션만 지원합니다.

의미론적 청킹과 계층적 청킹 기능(기존 옵션에 추가)을 통해 고객은 Lambda 함수를 사용하여 데이터 처리 및 청킹 방식을 더 잘 제어할 수 있습니다.
솔루션을 시연하기 위해 가상의 회사인 `Octank Financial`의 합성 10K 보고서를 데이터로 사용할 것입니다.
지식 베이스를 생성한 후 동일한 데이터셋에서 결과를 평가할 것입니다. 초점은 검색 결과의 품질을 개선하는 것이며, 이는 결과적으로 기초 모델이 생성하는 응답의 정확도를 향상시킬 것입니다.

## 1. 필요한 라이브러리 가져오기
첫 번째 단계는 사전 요구사항 패키지를 설치하는 것입니다.

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ragas==0.1.9 --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import botocore
botocore.__version__

'1.39.15'

In [5]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [6]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '211125368524')

In [7]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name_standard = 'standard-kb'
knowledge_base_name_hierarchical = 'hierarchical-kb'
knowledge_base_name_semantic = 'semantic-kb'
knowledge_base_name_custom = 'custom-chunking-kb'
knowledge_base_description = "Knowledge Base containing complex PDF."
bucket_name = f'{knowledge_base_name_standard}-{suffix}'
intermediate_bucket_name = f'{knowledge_base_name_standard}-intermediate-{suffix}'
lambda_function_name = f'{knowledge_base_name_custom}-lambda-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2. 고정 청킹 전략으로 지식 베이스 생성
[Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/)를 생성하여 레스토랑 메뉴를 저장하는 것부터 시작하겠습니다. Knowledge Bases는 [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise](https://console.harmony.a2z.com/internal-ai-assistant?playground=), [MongoDB Atlas](https://console.harmony.a2z.com/internal-ai-assistant?playground=) 등 다양한 벡터 데이터베이스와 통합할 수 있습니다. 이 예제에서는 지식 베이스를 Amazon OpenSearch Serverless와 통합할 것입니다. 이를 위해 BedrockKnowledgeBase 헬퍼 클래스를 사용하여 지식 베이스와 모든 사전 요구사항을 생성할 것입니다:
1. IAM 역할 및 정책
2. S3 버킷
3. Amazon OpenSearch Serverless 암호화, 네트워크 및 데이터 접근 정책
4. Amazon OpenSearch Serverless 컬렉션
5. Amazon OpenSearch Serverless 벡터 인덱스
6. 지식 베이스
7. 지식 베이스 데이터 소스

먼저 고정 청킹 전략을 사용하여 지식 베이스를 생성한 후, 계층적 청킹 전략을 사용하여 생성할 것입니다.

매개변수 값:

```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [8]:
knowledge_base_standard = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_standard}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-9072955']
buckets_to_check:  ['standard-kb-9072955']
Creating bucket standard-kb-9072955
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_9072955-f) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Tue, 29 Jul 2025 07:29:57 '
                                                 'GMT',
                                         'x-amzn-requestid': '31fd2f2d-7eea-41cd-8e87-0b4029b546e8'},
                        'HTTPStatusCode': 200,
                        'RequestId': '31fd2f2d-7ee

[2025-07-29 07:31:28,671] p27540 {base.py:258} INFO - PUT https://scampr9irtoyvp6tzd89.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-9072955-f [status:200 request:0.337s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-9072955-f',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 7, 29, 7, 32, 28, 799435, tzinfo=tzlocal()),
  'description': 'Knowledge Base containing complex PDF.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:211125368524:knowledge-base/C3DJFT63RJ',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'C3DJFT63RJ',
  'name': 'standard-kb-9072955',
  'roleArn': 'arn:aws:iam::211125368524:role/AmazonBedrockExecutionRoleForKnowledgeBase_9072955-f',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfigur

## 2.1 데이터셋을 Amazon S3에 업로드
이제 지식 베이스를 생성했으니, `Octank financial 10K` 보고서 데이터셋으로 채워보겠습니다. Knowledge Base 데이터 소스는 연결된 S3 버킷에서 데이터를 사용할 수 있어야 하며, `StartIngestionJob` 호출을 사용하여 데이터의 변경 사항을 지식 베이스와 동기화할 수 있습니다. 이 예제에서는 우리의 헬퍼 클래스를 통해 API의 [boto3 추상화](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html)를 사용할 것입니다.

먼저 dataset 폴더에 있는 메뉴 데이터를 s3에 업로드해보겠습니다.

In [10]:
import os

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("../synthetic_dataset", bucket_name)


uploading file ../synthetic_dataset/2024_population.pdf to standard-kb-9072955
uploading file ../synthetic_dataset/.ipynb_checkpoints/2024_population-checkpoint.pdf to standard-kb-9072955


이제 수집 작업을 시작하겠습니다.

In [11]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_standard.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'VDO2VETMRQ',
  'ingestionJobId': 'ZVAWDA3BRX',
  'knowledgeBaseId': 'C3DJFT63RJ',
  'startedAt': datetime.datetime(2025, 7, 29, 7, 33, 3, 251404, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 2,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 2},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 29, 7, 33, 28, 866035, tzinfo=tzlocal())}
........................................

마지막으로 나중에 솔루션을 테스트하기 위해 Knowledge Base Id를 저장합니다.

In [12]:
kb_id_standard = knowledge_base_standard.get_knowledge_base_id()

'C3DJFT63RJ'


### 2.2 지식 베이스 테스트
이제 지식 베이스를 사용할 수 있게 되었으므로 [retrieve](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용하여 테스트해볼 수 있습니다.

Retrieve and Generate API를 사용한 지식 베이스 테스트
먼저 retrieve and generate API를 사용하여 지식 베이스를 테스트해보겠습니다. 이 API를 사용하면 Bedrock이 지식 베이스에서 필요한 참조를 검색하고 Bedrock의 기초 모델을 사용하여 최종 답변을 생성하는 작업을 처리합니다.

query = 

`Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

`2019년 12월 31일로 종료된 회계연도의 Octank Financial의 연결 현금흐름표에 대한 요약을 제공하세요.`

실제 QA 쌍에 따른 이 쿼리에 대한 올바른 응답은 다음과 같습니다:

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. 
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
    
2019년 12월 31일로 종료된 연도의 Octank Financial 현금흐름표는 다음과 같습니다:
- 영업활동으로 인한 현금 창출액은 7억 1천만 달러로, 이는 7억 달러의 이익과 감가상각비 및 상각비와 같은 비현금성 지출에 기인합니다.
- 투자활동으로 인한 현금 유출액은 2억 4천만 달러로, 주요 지출은 유형자산 취득(2억 달러)과 시장성 있는 유가증권 취득(6천만 달러)이며, 이는 유형자산 매각(4천만 달러)과 만기 도래한 시장성 있는 유가증권(2천만 달러)으로 일부 상쇄되었습니다.
- 재무활동으로 인한 현금 유입액은 3억 5천만 달러로, 이는 보통주 발행(2억 달러)과 장기 부채 발행(3억 달러)에서 발생했으며, 보통주 자사주 매입(5천만 달러)과 장기 부채 상환(1억 달러)으로 현금 유입이 감소했습니다.
전반적으로 Octank Financial은 2019년에 1억 2천만 달러의 순현금 증가를 경험했으며, 이로 인해 총 현금 및 현금성 자산이 2억 1천만 달러가 되었습니다.
```

In [15]:
query = "2024년 주택의 노후도와 거주형태를 지역별, 유형별로 분석하고, 이를 인구・가구 특성과 연계하여 설명하시오."

In [16]:
time.sleep(20)
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_standard,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

2024년에 노후기간이 20년 이상 된 주택은 전체 주택의 54.9%를 차지하며, 30년 이상 된 주택은 28.0%를 차지할 것으로 예상됩니다. 주택 유형별로 보면 단독주택의 57.9%가 30년 이상 된 노후 주택이며, 아파트는 19.4%, 연립/다세대 주택은 25.7%, 비주거용 건물 내 주택은 47.7%가 30년 이상 된 노후 주택입니다. 지역별로는 전남(42.8%)과 경북(37.7%)이 30년 이상 된 노후 주택 비율이 가장 높고, 세종(7.5%)이 가장 낮습니다. 노후 주택 비율이 높은 지역은 인구 고령화와 가구 소규모화 추세와 연관될 수 있습니다. 반면 신규 주택 비율이 높은 지역은 젊은 인구와 대가구 비율이 상대적으로 높을 것으로 예상됩니다.



보시다시피, `RetreiveAndGenerate API`를 사용하면 최종 응답을 직접 얻을 수 있습니다. 이제 RetreiveAndGenerate API의 인용을 살펴보겠습니다. 이 노트북의 주요 초점은 모델이 응답을 생성하는 동안 검색된 청크들과 반환된 인용을 관찰하는 것입니다. 쿼리와 함께 관련 컨텍스트를 기초 모델에 제공하면, 높은 품질의 응답을 생성할 가능성이 높아집니다.

In [17]:
def citations_rag_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret,1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

In [18]:
response_standard = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_standard))
citations_rag_print(response_standard)

# of citations or chunks used to generate the response:  3
Chunk 1:  규모, 2024년      ❍ 노후기간이 20년 이상 된 주택은 10,908천호로 전체 주택의 54.9%를 차지하며,     30년 이상 된 주택은 5,574천호로 전체 주택의 28.0%임      - 노후기간이 30년 이상 된 주택의 비율은 전년 대비 2.2%p 증가, 5년 전     대비 9.9%p 증가     < 표 41 > 노후기간별 주택, 2019~2024년     (단위 : 천 호, %, %p)     주택종류 2019년 2020년 2021년 2022년 2023년 2024년 2019년 대비 2023년 대비     증감 증감률 증감 증감률     주     택      총주택 18,127 18,526 18,812 19,156

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-9072955/2024_population.pdf'}, 'type': 'S3'}

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-9072955/2024_population.pdf', 'x-amz-bedrock-kb-document-page-number': 73.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AkbMZVZgBBdp4KKBjr4yd', 'x-amz-bedrock-kb-data-source-id': 'VDO2VETMRQ'}

Chunk 2:  57.9%(2,226천호),     아파트는 19.4%(2,517천호)를 차지함      - 연립ㆍ다세대 주택과 비주거용 건물 내 주택은 30년 이상 된 주택     비율이 각각 25.7%(731천 호), 47.7%(101천 호) 임     < 표 42 > 노후기간 및 주택종류별 주택, 2024년     (단위 : 천 호, %)     노후기간 총주택 

이제 retrieve API를 사용하여 지식 베이스의 소스 정보를 검사해보겠습니다.

#### Retrieve API를 사용한 지식 베이스 테스트
추가적인 제어 계층이 필요한 경우, retrieve API를 사용하여 쿼리와 가장 잘 일치하는 청크들을 검색할 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. 그러면 API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수 및 청크 메타데이터를 제공합니다.

In [19]:
def response_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)


In [20]:
response_standard_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_standard, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)

print("# of retrieved results: ", len(response_standard_ret['retrievalResults']))
response_print(response_standard_ret)

# of retrieved results:  5
Chunk 1:  주택 구성비, 2024년     54.9     75.9     48.4 54.8     72.8     28.0     57.9     19.4 25.7     47.7     0.0     20.0     40.0     60.0     0.0     20.0     40.0     60.0     80.0     총 주택 단독주택 아파트 연립/다세대 비주거용     건물내주택     (%) 20년 이상 30년 이상     - 65 -❍ 시도별 노후기간이 30년 이상 된 주택 비율은 전남 42.8%(362천호), 경북     37.7%(426천호) 순으로 높으며, 세종이 7.5%(12천 호)로 가장 낮음     < 표 43 > 시도 및 노후기간별 주택, 2024년 (단위 : 천 호, %)     시 도

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-9072955/2024_population-checkpoint.pdf'}, 'type': 'S3'}

Chunk 1 Score:  0.6011895

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-9072955/2024_population-checkpoint.pdf', 'x-amz-bedrock-kb-document-page-number': 74.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AwrMZVZgBBdp4KKBjuY3U', 'x-amz-bedrock-kb-data-source-id': 'VDO2VETMRQ'}

Chunk 2:  주택 구성비, 2024년     54.9     75.9     48.4 54.8     72.8     28.0     57.9     19.4 25.7     47.7     0.0     20.0     40.0     60.0     0.0  

보시다시피, `fixed chunking(고정 청킹)`을 사용하면 `Retrieve API`의 기본값인 `semantic similarity(의미적 유사도)`를 사용하여 API에서 요청한 대로 5개의 검색 결과를 얻습니다. 이제 `hierarchical chunking(계층적 청킹)` 전략을 사용하고 `RetrieveAndGenerate API`와 `Retrieve API`를 모두 사용하여 검색된 결과를 검사해보겠습니다.

## 3. Hierarchical chunking(계층적 청킹) 전략으로 지식 베이스 생성

**개념**

Hierarchical chunking(계층적 청킹): 데이터를 계층적 구조로 구성하여 데이터 내의 고유한 관계를 기반으로 더 세분화되고 효율적인 검색이 가능하게 합니다. 데이터를 계층적 구조로 구성하면 RAG 워크플로우가 복잡하고 중첩된 데이터셋에서 정보를 효율적으로 탐색하고 검색할 수 있습니다.
문서가 파싱된 후, 첫 번째 단계는 상위 및 하위 청킹 크기를 기반으로 문서를 청크로 나누는 것입니다. 그런 다음 청크들은 계층적 구조로 구성되며, 상위 청크(높은 수준)는 더 큰 청크(예: 문서 또는 섹션)를 나타내고, 하위 청크(낮은 수준)는 더 작은 청크(예: 단락 또는 문장)를 나타냅니다. 상위와 하위 청크 간의 관계가 유지됩니다. 이러한 계층적 구조를 통해 코퍼스의 효율적인 검색과 탐색이 가능합니다.

**이점:**

효율적인 검색: 계층적 구조를 통해 관련 정보를 더 빠르고 더 타겟팅된 방식으로 검색할 수 있습니다. 먼저 하위 청크에서 의미론적 검색을 수행한 다음 검색 시 상위 청크를 반환합니다. 하위 청크를 상위 청크로 대체함으로써 FM에 더 크고 포괄적인 컨텍스트를 제공합니다.
컨텍스트 보존: 코퍼스를 계층적으로 구성하면 청크 간의 맥락적 관계가 보존되어 일관성 있고 맥락적으로 관련된 텍스트를 생성하는 데 도움이 될 수 있습니다.
><br>
>참고:
>계층적 청킹에서는 상위 청크가 반환되고 하위 청크에서 검색이 수행되므로, 하나의 상위가 여러 하위를 가질 수 있기 때문에 더 적은 수의 검색 결과가 반환될 수 있습니다.
><br></br>

계층적 청킹은 기술 매뉴얼, 법률 문서 또는 복잡한 형식과 중첩된 표가 있는 학술 논문과 같이 중첩되거나 계층적 구조를 가진 복잡한 문서에 가장 적합합니다.

**매개변수 값:**
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [21]:
knowledge_base_hierarchical = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_hierarchical}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "HIERARCHICAL", 
    suffix = f'{suffix}-h'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-9072955']
buckets_to_check:  ['standard-kb-9072955']
Bucket standard-kb-9072955 already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_9072955-h) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Tue, 29 Jul 2025 07:37:06 '
                                                 'GMT',
                                         'x-amzn-requestid': '03cd3a79-01ad-410f-8f75-2908289a5731'},
                        'HTTPStatusCode': 200,
                        'Re

[2025-07-29 07:38:36,968] p27540 {base.py:258} INFO - PUT https://djpat26jzszfkzcdbuw8.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-9072955-h [status:200 request:0.383s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-9072955-h',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is HIERARCHICAL
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 7, 29, 7, 39, 37, 96381, tzinfo=tzlocal()),
  'description': 'Knowledge Base containing complex PDF.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:211125368524:knowledge-base/OOVXFHUKUA',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'OOVXFHUKUA',
  'name': 'hierarchical-kb-9072955',
  'roleArn': 'arn:aws:iam::211125368524:role/AmazonBedrockExecutionRoleForKnowledgeBase_9072955-h',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessCon

이제 수집 작업을 시작하겠습니다. 고정 청킹에서 사용한 것과 동일한 문서를 사용하고 있으므로, s3 버킷에 문서를 업로드하는 단계는 건너뛰겠습니다.

In [22]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_hierarchical.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'NFTSTADCMW',
  'ingestionJobId': 'SZHQQXKASB',
  'knowledgeBaseId': 'OOVXFHUKUA',
  'startedAt': datetime.datetime(2025, 7, 29, 7, 40, 10, 220855, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 2,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 2},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 29, 7, 40, 39, 176090, tzinfo=tzlocal())}
........................................

추후 테스트를 위해 지식 베이스 ID를 저장합니다.

In [23]:
kb_id_hierarchical = knowledge_base_hierarchical.get_knowledge_base_id()

'OOVXFHUKUA'


### 3.1 지식 베이스 테스트
이제 지식 베이스를 사용할 수 있게 되었으므로 [retrieve](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용하여 테스트해볼 수 있습니다.

Retrieve and Generate API를 사용한 지식 베이스 테스트
먼저 retrieve and generate API를 사용하여 지식 베이스를 테스트해보겠습니다. 이 API를 사용하면 Bedrock이 지식 베이스에서 필요한 참조를 검색하고 Bedrock의 기초 모델을 사용하여 최종 답변을 생성하는 작업을 처리합니다.

In [24]:
time.sleep(20)
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_hierarchical,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

2024년 기준 전국 주택의 54.9%가 20년 이상 노후주택이며, 28.0%는 30년 이상 노후주택입니다. 단독주택의 경우 57.9%가 30년 이상 노후주택인 반면, 아파트는 19.4%만이 30년 이상 노후주택입니다. 지역별로는 전남(42.8%), 경북(37.7%), 대전(36.5%) 순으로 30년 이상 노후주택 비율이 높습니다. 1인 가구는 804만 가구로 전체 가구의 36.1%를 차지하며, 서울(39.9%)이 가장 높고 울산(31.6%)이 가장 낮습니다. 1인 가구의 연령대별 비중은 20대 이하(17.8%), 60대(17.6%), 30대(17.4%) 순입니다. 다문화 가구는 43.9만 가구로 지속 증가 추세이며, 경기(31.1%), 서울(17.8%) 순으로 비중이 높습니다. 장애인 인구는 261만 명으로 내국인의 5.2%이며, 장애인 가구는 229만 가구입니다.



As you can see, with the `RetreiveAndGenerate API` we get the final response directly, now let's observe the citations for `RetreiveAndGenerate API`. Since, our primary focus on this notebook is to observe the retrieved chunks and citations returned by the model while generating the response. When we provide the relevant context to the foundation model alongwith the query, it will most likely generate the high quality response.

In [25]:
response_hierarchical = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_hierarchical))
citations_rag_print(response_hierarchical)

# of citations or chunks used to generate the response:  2
Chunk 1:  호), 47.7%(101천 호) 임     < 표 42 > 노후기간 및 주택종류별 주택, 2024년     (단위 : 천 호, %)     노후기간 총주택 단독주택 아파트 연립/다세대 비주거용     건물내주택     주     택      총주택 19,873 3,841 12,974 2,846 211      20년 이상 10,908 2,914 6,280 1,560 154     20~30년 미만 5,334 688 3,763 829 53     30년 이상 5,574 2,226 2,517 731 101     구     성     비      총주택 100.0 100.0 100.0 100.0 100.0      20년 이상 54.9 75.9 48.4 54.8 72.8     20~30년 미만 26.8 17.9 29.0 29.1 25.1     30년 이상 28.0 57.9 19.4 25.7 47.7     * 노후기간은 11월 1일 기준으로 산정한 기간임     < 그림 48 > 노후기간 및 주택종류별 주택 구성비, 2024년     54.9     75.9     48.4 54.8     72.8     28.0     57.9     19.4 25.7     47.7     0.0     20.0     40.0     60.0     0.0     20.0     40.0     60.0     80.0     총 주택 단독주택 아파트 연립/다세대 비주거용     건물내주택     (%) 20년 이상 30년 이상     - 65 -❍ 시도별 노후기간이 30년 이상 된 주택 비율은 전남 42.8%(362천호), 경북     37.7%(426천호) 순으로 높으며, 세종이 7.5%(12천 호)로 가장 낮음     < 표 43 > 시도 및 노후기간별 주택, 2024년 (단위 : 천 호, %)     시 도 총주택 20년 이상 된 주택 30년

이제 retrieve API를 사용하여 지식 베이스에서 소스 정보를 검색해보겠습니다.

#### Retrieve API를 사용한 지식 베이스 테스트
만약 추가적인 제어 계층이 필요한 경우, retrieve API를 사용하여 쿼리와 가장 잘 일치하는 청크들을 검색할 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. 그러면 API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수 및 청크 메타데이터를 제공합니다.

In [26]:
response_hierarchical_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_hierarchical, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)

print("# of retrieved results: ", len(response_hierarchical_ret['retrievalResults']))
response_print(response_hierarchical_ret)

# of retrieved results:  3
Chunk 1:  호), 47.7%(101천 호) 임     < 표 42 > 노후기간 및 주택종류별 주택, 2024년     (단위 : 천 호, %)     노후기간 총주택 단독주택 아파트 연립/다세대 비주거용     건물내주택     주     택      총주택 19,873 3,841 12,974 2,846 211      20년 이상 10,908 2,914 6,280 1,560 154     20~30년 미만 5,334 688 3,763 829 53     30년 이상 5,574 2,226 2,517 731 101     구     성     비      총주택 100.0 100.0 100.0 100.0 100.0      20년 이상 54.9 75.9 48.4 54.8 72.8     20~30년 미만 26.8 17.9 29.0 29.1 25.1     30년 이상 28.0 57.9 19.4 25.7 47.7     * 노후기간은 11월 1일 기준으로 산정한 기간임     < 그림 48 > 노후기간 및 주택종류별 주택 구성비, 2024년     54.9     75.9     48.4 54.8     72.8     28.0     57.9     19.4 25.7     47.7     0.0     20.0     40.0     60.0     0.0     20.0     40.0     60.0     80.0     총 주택 단독주택 아파트 연립/다세대 비주거용     건물내주택     (%) 20년 이상 30년 이상     - 65 -❍ 시도별 노후기간이 30년 이상 된 주택 비율은 전남 42.8%(362천호), 경북     37.7%(426천호) 순으로 높으며, 세종이 7.5%(12천 호)로 가장 낮음     < 표 43 > 시도 및 노후기간별 주택, 2024년 (단위 : 천 호, %)     시 도 총주택 20년 이상 된 주택 30년 이상 된 주택     비율* 비율*     전 국 19,

><br>
>참고:
>위의 응답에서 볼 수 있듯이, retrieve API는 요청에서 5개를 지정했음에도 불구하고 3개의 검색 결과 또는 청크만 반환했습니다. 그 이유는 계층적 청킹에서는 API가 상위 청크를 반환하는 반면 검색은 하위 청크에서 수행되며, 하나의 상위 청크가 여러 개의 하위 청크를 가질 수 있기 때문입니다. 따라서 5개의 하위 청크에서 검색이 수행되었지만 응답은 3개의 청크만 반환되었습니다.
><br></br>

## 4. Semantic chunking(의미론적 청킹) 전략으로 지식 베이스 생성

**개념**

Semantic chunking(의미론적 청킹)은 텍스트 내의 관계를 분석하고 임베딩 모델이 계산한 의미적 유사도를 기반으로 의미 있고 완전한 청크로 나눕니다. 이 접근 방식은 검색 중에 정보의 무결성을 보존하여 정확하고 문맥적으로 적절한 결과를 보장하는 데 도움이 됩니다.
Amazon Bedrock Knowledge Bases는 먼저 지정된 토큰 크기를 기반으로 문서를 청크로 나눕니다. 각 청크에 대한 임베딩이 생성되고, 유사도 임계값과 버퍼 크기를 기반으로 임베딩 공간에서 유사한 청크들이 결합되어 새로운 청크를 형성합니다. 결과적으로 청크 크기는 청크마다 다를 수 있습니다.

**이점**

텍스트의 의미와 문맥에 초점을 맞춤으로써 의미론적 청킹은 검색 품질을 크게 향상시킵니다. 텍스트의 의미적 무결성을 유지하는 것이 중요한 시나리오에서 사용해야 합니다.

이 방법은 고정 크기 청킹보다 계산 집약적이지만, 법률 문서나 기술 매뉴얼과 같이 문맥적 경계가 명확하지 않은 문서를 청킹하는 데 유용할 수 있습니다.[[1]](#https://www.mongodb.com/developer/products/atlas/choosing-chunking-strategy-rag/)

**매개변수 값:**

```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [27]:
knowledge_base_semantic = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_semantic}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source, 
    chunking_strategy = "SEMANTIC", 
    suffix = f'{suffix}-s'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-9072955']
buckets_to_check:  ['standard-kb-9072955']
Bucket standard-kb-9072955 already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_9072955-s) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Tue, 29 Jul 2025 07:44:36 '
                                                 'GMT',
                                         'x-amzn-requestid': 'fe720982-314e-43ae-8fec-be654cde4dc3'},
                        'HTTPStatusCode': 200,
                        'Re

[2025-07-29 07:46:06,990] p27540 {base.py:258} INFO - PUT https://nd48li66unjz2tgzzuzb.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-9072955-s [status:200 request:0.371s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-9072955-s',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is SEMANTIC
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 7, 29, 7, 47, 7, 121511, tzinfo=tzlocal()),
  'description': 'Knowledge Base containing complex PDF.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:211125368524:knowledge-base/NY2KNVB6RT',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'NY2KNVB6RT',
  'name': 'semantic-kb-9072955',
  'roleArn': 'arn:aws:iam::211125368524:role/AmazonBedrockExecutionRoleForKnowledgeBase_9072955-s',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfigurati

이제 수집 작업을 시작하겠습니다. 고정 청킹에서 사용한 것과 동일한 문서를 사용하고 있으므로, s3 버킷에 문서를 업로드하는 단계는 건너뛰겠습니다.

In [28]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_semantic.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'TCNOAGDQN3',
  'ingestionJobId': 'UHI8HPJRNU',
  'knowledgeBaseId': 'NY2KNVB6RT',
  'startedAt': datetime.datetime(2025, 7, 29, 7, 47, 40, 20129, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 2,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 2},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 29, 7, 48, 10, 873085, tzinfo=tzlocal())}
........................................

In [29]:
kb_id_semantic = knowledge_base_semantic.get_knowledge_base_id()

'NY2KNVB6RT'


### 4.1 지식 베이스 테스트
이제 지식 베이스를 사용할 수 있게 되었으므로 [retrieve](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용하여 테스트해볼 수 있습니다.

#### Retrieve and Generate API를 사용한 지식 베이스 테스트
먼저 retrieve and generate API를 사용하여 지식 베이스를 테스트해보겠습니다. 이 API를 사용하면 Bedrock이 지식 베이스에서 필요한 참조를 검색하고 Bedrock의 기초 모델을 사용하여 최종 답변을 생성하는 작업을 처리합니다.

query = 

`Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

`2019년 12월 31일로 종료된 회계연도의 Octank Financial의 연결 현금흐름표에 대한 요약을 제공하세요.`

실제 QA 쌍에 따른 이 쿼리에 대한 올바른 응답은 다음과 같습니다:

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. 
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```
```
2019년 12월 31일로 종료된 연도의 Octank Financial 현금흐름표는 다음과 같습니다:
- 영업활동으로 인한 현금 창출액은 7억 1천만 달러로, 이는 7억 달러의 이익과 감가상각비 및 상각비와 같은 비현금성 지출에 기인합니다.
- 투자활동으로 인한 현금 유출액은 2억 4천만 달러로, 주요 지출은 유형자산 취득(2억 달러)과 시장성 있는 유가증권 취득(6천만 달러)이며, 이는 유형자산 매각(4천만 달러)과 만기 도래한 시장성 있는 유가증권(2천만 달러)으로 일부 상쇄되었습니다.
- 재무활동으로 인한 현금 유입액은 3억 5천만 달러로, 이는 보통주 발행(2억 달러)과 장기 부채 발행(3억 달러)에서 발생했으며, 보통주 자사주 매입(5천만 달러)과 장기 부채 상환(1억 달러)으로 현금 유입이 감소했습니다.
전반적으로 Octank Financial은 2019년에 1억 2천만 달러의 순현금 증가를 경험했으며, 이로 인해 총 현금 및 현금성 자산이 2억 1천만 달러가 되었습니다.
```

In [30]:
time.sleep(20)

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_semantic,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

2024년 기준으로 노후기간이 30년 이상 된 주택은 전체 주택의 28.0%를 차지합니다. 주택 유형별로는 단독주택이 57.9%로 가장 높은 비율을 보이며, 아파트는 19.4%, 연립/다세대 주택은 25.7%, 비주거용 건물 내 주택은 47.7%입니다. 지역별로는 전남(42.8%)과 경북(37.7%)이 30년 이상 노후주택 비율이 가장 높고, 세종(7.5%)이 가장 낮습니다. 노후주택 비율이 높은 지역은 인구 고령화와 가구 소규모화 추세가 두드러지는 경향이 있습니다.



보시다시피, `RetrieveAndGenerate API`를 사용하면 최종 응답을 직접 얻을 수 있습니다. 이제 `RetrieveAndGenerate API`의 인용을 살펴보겠습니다. 이 노트북의 주요 초점은 모델이 응답을 생성하는 동안 검색된 청크들과 반환된 인용을 관찰하는 것입니다. 쿼리와 함께 관련 컨텍스트를 기초 모델에 제공하면, 높은 품질의 응답을 생성할 가능성이 높아집니다.

In [31]:
response_semantic = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_semantic))
citations_rag_print(response_semantic)

# of citations or chunks used to generate the response:  2
Chunk 1:  20~30년 미만 29.9 29.7 29.0 28.8 27.9 26.8 -3.0 - -1.0 - 
 
 30년 이상 18.2 19.4 21.1 23.5 25.8 28.0 9.9 　- 2.2 - 
 
 * 노후기간은 11월 1일 기준으로 산정한 기간임 
 
 - 64 -❍ 주택종류별 노후기간이 30년 이상 된 주택은 단독주택이 57.9%(2,226천호), 
 
 아파트는 19.4%(2,517천호)를 차지함 
 
  - 연립ㆍ다세대 주택과 비주거용 건물 내 주택은 30년 이상 된 주택 
 
 비율이 각각 25.7%(731천 호), 47.7%(101천 호) 임 
 
 < 표 42 > 노후기간 및 주택종류별 주택, 2024년

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-9072955/2024_population.pdf'}, 'type': 'S3'}

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-9072955/2024_population.pdf', 'x-amz-bedrock-kb-document-page-number': 73.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AhLEnVZgBTpLj2rbkKKi0', 'x-amz-bedrock-kb-data-source-id': 'TCNOAGDQN3'}

Chunk 2:  20~30년 미만 29.9 29.7 29.0 28.8 27.9 26.8 -3.0 - -1.0 - 
 
 30년 이상 18.2 19.4 21.1 23.5 25.8 28.0 9.9 　- 2.2 - 
 
 * 노후기간은 11월 1일 기준으로 산정한 기간임 
 
 - 64 -❍ 주택종류별 노후기간이 30년 이상 된 주택은 단독주택이 57.9%(2,226천호), 
 
 아파트는

이제 retrieve API를 사용하여 지식 베이스에서 소스 정보를 검색해보겠습니다.

#### Retrieve API를 사용한 지식 베이스 테스트
만약 추가적인 제어 계층이 필요한 경우, retrieve API를 사용하여 쿼리와 가장 잘 일치하는 청크들을 검색할 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. 그러면 API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수 및 청크 메타데이터를 제공합니다.

In [32]:
response_semantic_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_semantic, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)
print("# of citations or chunks used to generate the response: ", len(response_semantic_ret['retrievalResults']))
response_print(response_semantic_ret)

# of citations or chunks used to generate the response:  5
Chunk 1:  20~30년 미만 29.9 29.7 29.0 28.8 27.9 26.8 -3.0 - -1.0 - 
 
 30년 이상 18.2 19.4 21.1 23.5 25.8 28.0 9.9 　- 2.2 - 
 
 * 노후기간은 11월 1일 기준으로 산정한 기간임 
 
 - 64 -❍ 주택종류별 노후기간이 30년 이상 된 주택은 단독주택이 57.9%(2,226천호), 
 
 아파트는 19.4%(2,517천호)를 차지함 
 
  - 연립ㆍ다세대 주택과 비주거용 건물 내 주택은 30년 이상 된 주택 
 
 비율이 각각 25.7%(731천 호), 47.7%(101천 호) 임 
 
 < 표 42 > 노후기간 및 주택종류별 주택, 2024년

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-9072955/2024_population-checkpoint.pdf'}, 'type': 'S3'}

Chunk 1 Score:  0.60518813

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-9072955/2024_population-checkpoint.pdf', 'x-amz-bedrock-kb-document-page-number': 73.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AcLEnVZgBTpLj2rbkMalj', 'x-amz-bedrock-kb-data-source-id': 'TCNOAGDQN3'}

Chunk 2:  20~30년 미만 29.9 29.7 29.0 28.8 27.9 26.8 -3.0 - -1.0 - 
 
 30년 이상 18.2 19.4 21.1 23.5 25.8 28.0 9.9 　- 2.2 - 
 
 * 노후기간은 11월 1일 기준으로 산정한 기간임 
 
 - 64 -❍ 주택종류별

## 5. Lambda 함수를 사용한 사용자 정의 청킹 옵션
Amazon Bedrock용 Knowledge Bases(KB)를 생성할 때 Lambda 함수를 연결하여 사용자 정의 청킹 로직을 지정할 수 있습니다. 수집 과정에서 lambda 함수가 제공되면, Knowledge Bases는 lambda 함수를 실행하고 입력 및 출력 값을 제공된 중간 s3 버킷에 저장합니다.

><br>
>참고: KB와 함께 사용되는 Lambda 함수는 사용자 정의 청킹 로직 추가뿐만 아니라 청크 수준 메타데이터 추가와 같은 청크 처리에도 사용될 수 있습니다. 이 예제에서는 사용자 정의 청킹 로직을 위한 Lambda 함수 사용에 초점을 맞추고 있습니다.
><br></br>

### 5.1 Lambda 함수 생성

이제 사용자 정의 청킹을 위한 코드가 포함된 lambda 함수를 생성할 것입니다. 이를 위해 다음을 수행합니다:

1. 사용자 정의 청킹을 위한 로직이 포함된 lambda_function.py 파일을 생성합니다.
2. Lambda 함수를 위한 IAM 역할을 생성합니다.
3. 필요한 권한을 가진 lambda 함수를 생성합니다.

#### Create the function code
 Let's create the lambda function tha implements the functions for `reading your file from intermediate bucket`, `process the contents with custom chunking logic` and `write the output back to s3 bucket`. 
#### 함수 코드 생성
`reading your file from intermediate bucket`, `process the contents with custom chunking logic`, `write the output back to s3 bucket` 기능을 구현하는 lambda 함수를 생성해 보겠습니다.

In [33]:
%%writefile lambda_function.py
import json
from abc import abstractmethod, ABC
from typing import List
from urllib.parse import urlparse
import boto3
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

class Chunker(ABC):
    @abstractmethod
    def chunk(self, text: str) -> List[str]:
        raise NotImplementedError()
        
class SimpleChunker(Chunker):
    def chunk(self, text: str) -> List[str]:
        words = text.split()
        return [' '.join(words[i:i+100]) for i in range(0, len(words), 100)]

def lambda_handler(event, context):
    logger.debug('input={}'.format(json.dumps(event)))
    s3 = boto3.client('s3')

    # Extract relevant information from the input event
    input_files = event.get('inputFiles')
    input_bucket =  event.get('bucketName')

    
    if not all([input_files, input_bucket]):
        raise ValueError("Missing required input parameters")
    
    output_files = []
    chunker = SimpleChunker()

    for input_file in input_files:
        content_batches = input_file.get('contentBatches', [])
        file_metadata = input_file.get('fileMetadata', {})
        original_file_location = input_file.get('originalFileLocation', {})

        processed_batches = []
        
        for batch in content_batches:
            input_key = batch.get('key')

            if not input_key:
                raise ValueError("Missing uri in content batch")
            
            # Read file from S3
            file_content = read_s3_file(s3, input_bucket, input_key)
            
            # Process content (chunking)
            chunked_content = process_content(file_content, chunker)
            
            output_key = f"Output/{input_key}"
            
            # Write processed content back to S3
            write_to_s3(s3, input_bucket, output_key, chunked_content)
            
            # Add processed batch information
            processed_batches.append({
                'key': output_key
            })
        
        # Prepare output file information
        output_file = {
            'originalFileLocation': original_file_location,
            'fileMetadata': file_metadata,
            'contentBatches': processed_batches
        }
        output_files.append(output_file)
    
    result = {'outputFiles': output_files}
    
    return result
    

def read_s3_file(s3_client, bucket, key):
    response = s3_client.get_object(Bucket=bucket, Key=key)
    return json.loads(response['Body'].read().decode('utf-8'))

def write_to_s3(s3_client, bucket, key, content):
    s3_client.put_object(Bucket=bucket, Key=key, Body=json.dumps(content))    

def process_content(file_content: dict, chunker: Chunker) -> dict:
    chunked_content = {
        'fileContents': []
    }
    
    for content in file_content.get('fileContents', []):
        content_body = content.get('contentBody', '')
        content_type = content.get('contentType', '')
        content_metadata = content.get('contentMetadata', {})
        
        words = content['contentBody']
        chunks = chunker.chunk(words)
        
        for chunk in chunks:
            chunked_content['fileContents'].append({
                'contentType': content_type,
                'contentMetadata': content_metadata,
                'contentBody': chunk
            })
    
    return chunked_content

Writing lambda_function.py


지식 베이스가 제공하는 표준 청킹 전략 값은 다음과 같습니다:

**매개변수 값:**

```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```
    
사용자 정의 로직을 구현하기 위해 knowledge_base.py 클래스에 CUSTOM 값을 전달하는 옵션을 포함했습니다.
이 클래스에서 청킹 전략을 CUSTOM으로 전달하면 다음과 같이 동작합니다:

1. chunkingStrategy를 NONE으로 선택합니다.
2. vectorIngestionConfiguration에 다음과 같이 customTransformationConfiguration을 추가합니다:
    
```
{
...
   "vectorIngestionConfiguration": {
    "customTransformationConfiguration": { 
         "intermediateStorage": { 
            "s3Location": { 
               "uri": "string"
            }
         },
         "transformations": [
            {
               "transformationFunction": {
                  "lambdaConfiguration": {
                     "lambdaArn": "string"
                  }
               },
               "stepToApply": "string" // enum of POST_CHUNKING
            }
         ]
      },
      "chunkingConfiguration": {
         "chunkingStrategy": "NONE"
         ...
   }
}

```

In [34]:
knowledge_base_custom = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_custom}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    lambda_function_name=lambda_function_name,
    intermediate_bucket_name=intermediate_bucket_name, 
    chunking_strategy = "CUSTOM", 
    suffix = f'{suffix}-c'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['standard-kb-9072955', 'standard-kb-intermediate-9072955']
buckets_to_check:  ['standard-kb-9072955', 'standard-kb-intermediate-9072955']
Bucket standard-kb-9072955 already exists - retrieving it!
Creating bucket standard-kb-intermediate-9072955
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_9072955-c) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Tue, 29 Jul 2025 07:49:22 '
                                                 'GMT',
                                         'x-amzn-reques

[2025-07-29 07:50:52,827] p27540 {base.py:258} INFO - PUT https://towlurvhc9pkv2602zkh.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-9072955-c [status:200 request:0.373s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-9072955-c',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Creating lambda function... as chunking strategy is CUSTOM
{'ResponseMetadata': {'RequestId': '2a5dd03b-c040-4aaa-9aea-8b1b16b8591a', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 29 Jul 2025 07:52:03 GMT', 'content-type': 'application/json', 'content-length': '1420', 'connection': 'keep-alive', 'x-amzn-requestid': '2a5dd03b-c040-4aaa-9aea-8b1b16b8591a'}, 'RetryAttempts': 0}, 'FunctionName': 'custom-chunking-kb-lambda-9072955', 'FunctionArn': 'arn:aws:lambda:us-west-2:211125368524:function:custom-chunking-kb-lambda-9072955', 'Runtime': 'python3.12', 'Role': 'arn:aws:iam::211125368524:role/custom-chunking-kb-9072955-lambda-role-9072955-c', 'Handler': 'lambda_function.lambda_handler', 'CodeSize': 3465, 'Description': '', 'Timeout': 60, 'MemorySize': 128, 'LastModified': '2025-07-29T07:5

이제 수집 작업을 시작하겠습니다.

In [54]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_custom.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'EPVYU4TOPB',
  'ingestionJobId': 'X2HP25PTRQ',
  'knowledgeBaseId': 'UADADKRAXE',
  'startedAt': datetime.datetime(2025, 7, 29, 8, 40, 45, 168183, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 2,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 29, 8, 40, 45, 941997, tzinfo=tzlocal())}
........................................

In [55]:
kb_id_custom = knowledge_base_custom.get_knowledge_base_id()

'UADADKRAXE'


### 5.2 지식 베이스 테스트
이제 지식 베이스를 사용할 수 있게 되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용하여 테스트해볼 수 있습니다.

#### Retrieve and Generate API를 사용한 지식 베이스 테스트

먼저 retrieve and generate API를 사용하여 지식 베이스를 테스트해보겠습니다. 이 API를 사용하면 Bedrock이 지식 베이스에서 필요한 참조를 검색하고 Bedrock의 기초 모델을 사용하여 최종 답변을 생성하는 작업을 처리합니다.

query = 

`Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

`2019년 12월 31일로 종료된 회계연도의 Octank Financial의 연결 현금흐름표에 대한 요약을 제공하세요.`

실제 QA 쌍에 따른 이 쿼리에 대한 올바른 응답은 다음과 같습니다:

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow. 
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```
```
2019년 12월 31일로 종료된 연도의 Octank Financial 현금흐름표는 다음과 같습니다:
- 영업활동으로 인한 현금 창출액은 7억 1천만 달러로, 이는 7억 달러의 이익과 감가상각비 및 상각비와 같은 비현금성 지출에 기인합니다.
- 투자활동으로 인한 현금 유출액은 2억 4천만 달러로, 주요 지출은 유형자산 취득(2억 달러)과 시장성 있는 유가증권 취득(6천만 달러)이며, 이는 유형자산 매각(4천만 달러)과 만기 도래한 시장성 있는 유가증권(2천만 달러)으로 일부 상쇄되었습니다.
- 재무활동으로 인한 현금 유입액은 3억 5천만 달러로, 이는 보통주 발행(2억 달러)과 장기 부채 발행(3억 달러)에서 발생했으며, 보통주 자사주 매입(5천만 달러)과 장기 부채 상환(1억 달러)으로 현금 유입이 감소했습니다.
전반적으로 Octank Financial은 2019년에 1억 2천 2천만 달러의 순현금 증가를 경험했으며, 이로 인해 총 현금 및 현금성 자산이 2억 1천만 달러가 되었습니다.
```

In [56]:
time.sleep(10)

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_custom,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

2024년 기준으로 노후기간이 30년 이상 된 주택의 비율은 전남(42.8%), 경북(37.7%), 강원(32.4%), 경남(32.3%) 등 지방에서 높게 나타났습니다. 반면 세종(7.5%)과 경기(17.9%)는 상대적으로 낮은 비율을 보였습니다. 주택 유형별로는 단독주택이 57.9%로 가장 높은 노후율을 보였고, 아파트는 19.4%로 가장 낮았습니다. 이러한 지역별, 유형별 노후주택 비율의 차이는 지역의 인구 및 가구 특성과 연관될 수 있습니다. 대도시나 신도시 지역은 상대적으로 최근에 아파트 단지가 많이 건설되어 노후주택 비율이 낮은 반면, 농촌 지역은 오래된 단독주택이 많아 노후주택 비율이 높습니다. 또한 1인 가구나 고령 가구 비율이 높은 지역일수록 노후주택 거주 비율이 높을 것으로 예상됩니다.



보시다시피, `RetrieveAndGenerate API`를 사용하면 최종 응답을 직접 얻을 수 있습니다. 이제 `RetrieveAndGenerate API`의 인용을 살펴보겠습니다. 이 노트북의 주요 초점은 모델이 응답을 생성하는 동안 검색된 청크들과 반환된 인용을 관찰하는 것입니다. 쿼리와 함께 관련 컨텍스트를 기초 모델에 제공하면, 높은 품질의 응답을 생성할 가능성이 높아집니다.

In [57]:
response_custom = response['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_custom))
citations_rag_print(response_custom)

# of citations or chunks used to generate the response:  2
Chunk 1:  -❍ 시도별 노후기간이 30년 이상 된 주택 비율은 전남 42.8%(362천호), 경북 37.7%(426천호) 순으로 높으며, 세종이 7.5%(12천 호)로 가장 낮음 < 표 43 > 시도 및 노후기간별 주택, 2024년 (단위 : 천 호, %) 시 도 총주택 20년 이상 된 주택 30년 이상 된 주택 비율* 비율* 전 국 19,873 10,908 54.9 5,574 28.0 서 울 3,170 1,867 58.9 897 28.3 부 산 1,350 799 59.2 433 32.1 대 구 904 518 57.3 266 29.4 인 천 1,153 614 53.2 346 30.1 광 주 568 324 57.1 164 28.9 대 전 524 324 61.9 191 36.5 울 산 412 238 57.9

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-9072955/2024_population.pdf'}, 'type': 'S3'}

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-9072955/2024_population.pdf', 'x-amz-bedrock-kb-document-page-number': 1.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AQbMrVZgBBdp4KKBjjJJ_', 'x-amz-bedrock-kb-data-source-id': 'EPVYU4TOPB'}

Chunk 2:  52.2 53.7 54.9 6.9 - 1.2 - 20~30년 미만 29.9 29.7 29.0 28.8 27.9 26.8 -3.0 - -1.0 - 30년 이상 18.2 19.4 21.1 23.5 25.8 28.0 9.9 - 2.2 - * 노후기간은 11월 1일 기준으로 산정한 기간임 - 64 -❍

이제 retrieve API를 사용하여 지식 베이스에서 소스 정보를 검색해보겠습니다.

#### Retrieve API를 사용한 지식 베이스 테스트
만약 추가적인 제어 계층이 필요한 경우, retrieve API를 사용하여 쿼리와 가장 잘 일치하는 청크들을 검색할 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. 그러면 API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수 및 청크 메타데이터를 제공합니다.

In [58]:
response_custom_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_custom, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': query
    }
)
print("# of citations or chunks used to generate the response: ", len(response_custom_ret['retrievalResults']))
response_print(response_custom_ret)

# of citations or chunks used to generate the response:  5
Chunk 1:  52.2 53.7 54.9 6.9 - 1.2 - 20~30년 미만 29.9 29.7 29.0 28.8 27.9 26.8 -3.0 - -1.0 - 30년 이상 18.2 19.4 21.1 23.5 25.8 28.0 9.9 - 2.2 - * 노후기간은 11월 1일 기준으로 산정한 기간임 - 64 -❍ 주택종류별 노후기간이 30년 이상 된 주택은 단독주택이 57.9%(2,226천호), 아파트는 19.4%(2,517천호)를 차지함 - 연립ㆍ다세대 주택과 비주거용 건물 내 주택은 30년 이상 된 주택 비율이 각각 25.7%(731천 호), 47.7%(101천 호) 임 < 표 42 > 노후기간 및 주택종류별 주택, 2024년 (단위 : 천 호, %) 노후기간 총주택 단독주택 아파트 연립/다세대 비주거용 건물내주택 주 택 총주택 19,873 3,841 12,974 2,846 211 20년

Chunk 1 Location:  {'s3Location': {'uri': 's3://standard-kb-9072955/2024_population.pdf'}, 'type': 'S3'}

Chunk 1 Score:  0.57461923

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://standard-kb-9072955/2024_population.pdf', 'x-amz-bedrock-kb-document-page-number': 1.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AP7MrVZgBBdp4KKBjjJJ_', 'x-amz-bedrock-kb-data-source-id': 'EPVYU4TOPB'}

Chunk 2:  52.2 53.7 54.9 6.9 - 1.2 - 20~30년 미만 29.9 29.7 29.0 28.8 27.9 26.8 -3.0 - -1.0 - 30년 이상 18.

모든 경우에서 하나의 쿼리를 평가할 때 정확한 응답을 얻었습니다. 하지만 RAG 애플리케이션을 구축할 때는 정확도 향상을 파악하기 위해 많은 수의 질문과 답변으로 평가해야 합니다. 다음 단계에서는 오픈 소스 프레임워크인 RAG Assessment(RAGAS)를 사용하여 컨텍스트 또는 검색 결과의 품질을 평가하는 지표에 대해 `your dataset`에서 응답을 평가할 것입니다.
다음 2가지 지표에만 초점을 맞출 것입니다:

1. 컨텍스트 재현율(Context recall)
2. 컨텍스트 관련성(Context relevancy)

## 6. RAG Assessment(RAGAS) 프레임워크를 사용하여 데이터셋의 검색 결과 평가
RAGAS 프레임워크를 사용하여 각 청킹 전략에 대한 결과를 평가할 수 있습니다. 이 접근 방식은 귀하의 데이터셋에 어떤 청킹 전략을 사용할지에 대한 사실적인 지침을 제공하는 데 도움이 될 수 있습니다.

이상적으로는 다른 매개변수도 최적화를 고려해야 합니다. 예를 들어 계층적 청킹의 경우 상위 청크나 하위 청크에 대해 다른 크기를 시도해 보아야 합니다.

아래 접근 방식은 Amazon Bedrock Knowledge Bases가 권장하는 기본 매개변수를 기반으로 어떤 전략을 사용할 수 있는지에 대한 휴리스틱을 제공할 것입니다.

#### 평가
이 섹션에서는 RAGAS를 활용하여 다음 지표들을 사용해 검색 결과를 평가할 것입니다:
1. **컨텍스트 재현율(Context Recall)**: 컨텍스트 재현율은 검색된 컨텍스트가 실제 정답(ground truth)으로 취급되는 주석이 달린 답변과 얼마나 일치하는지를 측정합니다. 이는 실제 정답과 검색된 컨텍스트를 기반으로 계산되며, 값은 0에서 1 사이이고 높을수록 더 나은 성능을 나타냅니다.

2. **컨텍스트 관련성(Context relevancy)**: 이 지표는 검색된 컨텍스트의 관련성을 측정하며, 질문과 컨텍스트를 모두 기반으로 계산됩니다. 값은 (0, 1) 범위 내에 있으며, 높을수록 더 나은 관련성을 나타냅니다.

In [73]:
from datetime import datetime

# 모델 ID 설정
MODEL_ID_EVAL = "anthropic.claude-3-sonnet-20240229-v1:0"
MODEL_ID_GEN = "anthropic.claude-3-haiku-20240307-v1:0"

print(f"평가 모델: {MODEL_ID_EVAL}")
print(f"생성 모델: {MODEL_ID_GEN}")

# 평가 질문과 정답
questions = [
    "2024년 시도별 인구・가구・주택 특성을 종합적으로 비교 분석하시오.(단, 각 시도의 연령구조, 1인가구 비율, 주택 노후도, 외국인 비율을 모두 포함하여 상위 3개 지역과 하위 3개 지역을 각각 제시하고, 이들 간의 특성 차이를 설명하시오)",
]

ground_truths = [
    "2024년 시도별 특성을 종합 분석하면 다음과 같습니다:\
    고령화 측면(고령인구 비율)\
    상위: 전남(26.3%), 경북(24.9%), 강원(24.6%)\
    하위: 세종(11.1%), 울산(16.8%), 경기(16.3%)\
    1인가구 비율\
    상위: 서울(39.9%), 강원(39.4%), 대전(39.8%)\
    하위: 세종(32.9%), 울산(31.6%), 경기(31.7%)\
    주택 노후도(30년 이상)\
    상위: 전남(42.8%), 경북(37.7%), 대전(36.5%)\
    하위: 세종(7.5%), 경기(17.9%), 충남(25.7%)\
    외국인 비율\
    상위: 경기(4.9%), 서울(3.9%), 충남(6.2%)\
    하위: 대전(2.1%), 대구(1.9%), 부산(2.1%)\
    지역별 특성 분석:\
    대도시권\
    서울: 1인가구 비율(39.9%)과 외국인 비율(3.9%)이 높으나, 주택 노후도도 높음\
    경기: 외국인 비율이 높고(4.9%) 주택 노후도는 낮으나(17.9%), 1인가구 비율은 낮음(31.7%)\
    인천: 중간적 특성을 보이나 외국인 비율이 높음(4.2%)\
    신도시 특성\
    세종: 모든 지표에서 가장 낮은 수준(고령화율 11.1%, 주택노후도 7.5%)\
    울산: 1인가구 비율이 가장 낮고(31.6%) 고령화율도 낮음(16.8%)\
    농어촌 지역\
    전남: 고령화율(26.3%)과 주택노후도(42.8%) 모두 최고 수준\
    경북: 고령화율(24.9%)과 주택노후도(37.7%) 모두 높은 수준\
    지역간 격차의 주요 특성:\
    수도권-비수도권 격차\
    수도권: 외국인 비율이 높고 노후주택 비율이 낮음\
    비수도권: 고령화율과 노후주택 비율이 높음\
    도시-농촌 격차\
    도시지역: 1인가구 비율이 높고 주택노후도는 상대적으로 낮음\
    농촌지역: 고령화율과 주택노후도가 높음\
    신구도시 격차\
    신도시(세종 등): 모든 지표에서 양호한 수준\
    구도시(대전 등): 주택노후도와 1인가구 비율이 높음\
    이러한 지역별 특성은 각 지역의 산업구조, 개발시기, 인구이동 패턴 등이 복합적으로 작용한 결과로 볼 수 있습니다. 특히 수도권과 비수도권의 격차, 도시와 농촌의 격차가 뚜렷하게 나타나고 있으며, 이는 향후 지역별 맞춤형 정책 수립이 필요함을 시사합니다."
]

print(f"평가 질문 수: {len(questions)}")
print(f"정답 수: {len(ground_truths)}")

평가 모델: anthropic.claude-3-sonnet-20240229-v1:0
생성 모델: anthropic.claude-3-haiku-20240307-v1:0
평가 질문 수: 1
정답 수: 1


In [74]:
class BedrockRAGEvaluator:
    """
    Amazon Bedrock을 사용한 RAG 평가 클래스
    """
    
    def __init__(self, model_id_eval, model_id_generation, kb_id):
        self.model_id_eval = model_id_eval
        self.model_id_generation = model_id_generation
        self.kb_id = kb_id
        self.bedrock_client = boto3.client('bedrock-runtime')
        self.bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')
    
    def retrieve_and_generate(self, query):
        """Knowledge Base에서 답변 생성 및 컨텍스트 검색"""
        try:
            response = self.bedrock_agent_runtime.retrieve_and_generate(
                input={'text': query},
                retrieveAndGenerateConfiguration={
                    'type': 'KNOWLEDGE_BASE',
                    'knowledgeBaseConfiguration': {
                        'knowledgeBaseId': self.kb_id,
                        'modelArn': f'arn:aws:bedrock:us-west-2::foundation-model/{self.model_id_generation}',
                        'retrievalConfiguration': {
                            'vectorSearchConfiguration': {
                                'numberOfResults': 5
                            }
                        }
                    }
                }
            )
            
            generated_answer = response['output']['text']
            retrieved_contexts = []
            
            # 검색된 컨텍스트 추출
            if 'citations' in response:
                for citation in response['citations']:
                    if 'retrievedReferences' in citation:
                        for ref in citation['retrievedReferences']:
                            if 'content' in ref and 'text' in ref['content']:
                                retrieved_contexts.append(ref['content']['text'])
            
            return generated_answer, retrieved_contexts
            
        except Exception as e:
            print(f"검색 및 생성 중 오류: {str(e)}")
            return None, []

print("BedrockRAGEvaluator 클래스 정의 완료")

BedrockRAGEvaluator 클래스 정의 완료


In [75]:
def evaluate_context_recall(self, question, ground_truth, contexts):
    """컨텍스트 재현율 평가"""
    
    recall_prompt = f"""
    다음 질문에 대한 정답과 검색된 컨텍스트를 비교하여 컨텍스트 재현율(Context Recall)을 평가해주세요.

    질문: {question}
    
    정답 (Ground Truth): {ground_truth}
    
    검색된 컨텍스트들:
    {chr(10).join([f"{i+1}. {ctx}" for i, ctx in enumerate(contexts)])}
    
    컨텍스트 재현율은 정답에 포함된 정보 중 얼마나 많은 부분이 검색된 컨텍스트에서 찾을 수 있는지를 측정합니다.
    
    평가 기준:
    - 1.0: 정답의 모든 핵심 정보가 컨텍스트에 포함됨
    - 0.8: 정답의 대부분 정보가 컨텍스트에 포함됨
    - 0.6: 정답의 절반 정도 정보가 컨텍스트에 포함됨
    - 0.4: 정답의 일부 정보만 컨텍스트에 포함됨
    - 0.2: 정답의 매우 적은 정보가 컨텍스트에 포함됨
    - 0.0: 정답의 정보가 컨텍스트에 거의 포함되지 않음
    
    응답 형식 (JSON):
    {{
        "context_recall_score": 0.0-1.0,
        "missing_information": ["누락된 핵심 정보들"],
        "found_information": ["컨텍스트에서 찾은 정보들"],
        "explanation": "평가 근거 설명"
    }}
    """
    
    try:
        response = self.bedrock_client.invoke_model(
            modelId=self.model_id_eval,
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 1500,
                "messages": [{"role": "user", "content": recall_prompt}]
            })
        )
        
        result = json.loads(response['body'].read())
        return result['content'][0]['text']
        
    except Exception as e:
        return f"컨텍스트 재현율 평가 중 오류: {str(e)}"

# BedrockRAGEvaluator 클래스에 메서드 추가
BedrockRAGEvaluator.evaluate_context_recall = evaluate_context_recall

print("컨텍스트 재현율 평가 메서드 추가 완료")

컨텍스트 재현율 평가 메서드 추가 완료


In [76]:
def evaluate_context_precision(self, question, contexts):
    """컨텍스트 정밀도 평가"""
    
    precision_prompt = f"""
    다음 질문에 대해 검색된 컨텍스트들의 관련성을 평가하여 컨텍스트 정밀도(Context Precision)를 계산해주세요.

    질문: {question}
    
    검색된 컨텍스트들:
    {chr(10).join([f"{i+1}. {ctx}" for i, ctx in enumerate(contexts)])}
    
    컨텍스트 정밀도는 검색된 컨텍스트 중 질문과 관련된 컨텍스트의 비율을 측정합니다.
    
    각 컨텍스트에 대해 다음과 같이 평가해주세요:
    - 매우 관련성 높음 (1.0): 질문에 직접적으로 답변하는 정보 포함
    - 관련성 높음 (0.8): 질문과 밀접한 관련이 있는 정보 포함
    - 보통 관련성 (0.6): 질문과 어느 정도 관련된 정보 포함
    - 낮은 관련성 (0.4): 질문과 약간의 관련성만 있음
    - 매우 낮은 관련성 (0.2): 질문과 거의 관련 없음
    - 관련성 없음 (0.0): 질문과 전혀 관련 없음
    
    응답 형식 (JSON):
    {{
        "context_evaluations": [
            {{"context_id": 1, "relevance_score": 0.0-1.0, "explanation": "관련성 설명"}},
            {{"context_id": 2, "relevance_score": 0.0-1.0, "explanation": "관련성 설명"}}
        ],
        "overall_precision_score": 0.0-1.0,
        "explanation": "전체 정밀도 평가 설명"
    }}
    """
    
    try:
        response = self.bedrock_client.invoke_model(
            modelId=self.model_id_eval,
            body=json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 2000,
                "messages": [{"role": "user", "content": precision_prompt}]
            })
        )
        
        result = json.loads(response['body'].read())
        return result['content'][0]['text']
        
    except Exception as e:
        return f"컨텍스트 정밀도 평가 중 오류: {str(e)}"

# BedrockRAGEvaluator 클래스에 메서드 추가
BedrockRAGEvaluator.evaluate_context_precision = evaluate_context_precision

print("컨텍스트 정밀도 평가 메서드 추가 완료")

컨텍스트 정밀도 평가 메서드 추가 완료


In [77]:
def evaluate(self, questions, ground_truths):
    """전체 평가 수행"""
    
    evaluation_results = []
    
    for i, (question, ground_truth) in enumerate(zip(questions, ground_truths)):
        print(f"질문 {i+1} 평가 중...")
        
        # 답변 생성 및 컨텍스트 검색
        generated_answer, contexts = self.retrieve_and_generate(question)
        
        if generated_answer is None:
            evaluation_results.append({
                'question': question,
                'error': '답변 생성 실패'
            })
            continue
        
        # 컨텍스트 재현율 평가
        context_recall_result = self.evaluate_context_recall(question, ground_truth, contexts)
        
        # 컨텍스트 정밀도 평가
        context_precision_result = self.evaluate_context_precision(question, contexts)
        
        evaluation_results.append({
            'question': question,
            'generated_answer': generated_answer,
            'ground_truth': ground_truth,
            'contexts': contexts,
            'context_recall': context_recall_result,
            'context_precision': context_precision_result,
            'timestamp': datetime.now().isoformat()
        })
        
        # API 호출 제한을 위한 대기
        time.sleep(2)
    
    return evaluation_results

# BedrockRAGEvaluator 클래스에 메서드 추가
BedrockRAGEvaluator.evaluate = evaluate

print("전체 평가 메서드 추가 완료")

전체 평가 메서드 추가 완료


In [78]:
def run_chunking_strategy_evaluation(kb_ids, kb_names, questions, ground_truths):
    """
    다양한 청킹 전략에 대한 비교 평가 수행
    """
    
    all_results = {}
    
    for kb_id, kb_name in zip(kb_ids, kb_names):
        print(f"\n=== {kb_name} 청킹 전략 평가 시작 ===")
        
        evaluator = BedrockRAGEvaluator(
            model_id_eval=MODEL_ID_EVAL,
            model_id_generation=MODEL_ID_GEN,
            kb_id=kb_id
        )
        
        results = evaluator.evaluate(questions, ground_truths)
        all_results[kb_name] = results
        
        print(f"{kb_name} 평가 완료")
        time.sleep(5)  # Knowledge Base 간 전환 대기
    
    return all_results

print("청킹 전략 비교 평가 함수 정의 완료")

청킹 전략 비교 평가 함수 정의 완료


In [79]:
def extract_scores_from_results(results):
    """평가 결과에서 점수 추출"""
    
    scores = {
        'context_recall': [],
        'context_precision': []
    }
    
    for result in results:
        if 'error' not in result:
            # JSON 응답에서 점수 추출 시도
            try:
                recall_text = result['context_recall']
                precision_text = result['context_precision']
                
                # 간단한 점수 추출 (실제로는 더 정교한 파싱 필요)
                recall_match = re.search(r'"context_recall_score":\s*([0-9.]+)', recall_text)
                precision_match = re.search(r'"overall_precision_score":\s*([0-9.]+)', precision_text)
                
                if recall_match:
                    scores['context_recall'].append(float(recall_match.group(1)))
                if precision_match:
                    scores['context_precision'].append(float(precision_match.group(1)))
                    
            except Exception as e:
                print(f"점수 추출 중 오류: {str(e)}")
    
    return scores

print("점수 추출 함수 정의 완료")

점수 추출 함수 정의 완료


In [80]:
# Knowledge Base ID들 (실제 값으로 교체 필요)
# 이 값들은 앞서 생성된 Knowledge Base들의 실제 ID로 교체해야 합니다.
print("Semantic: ", kb_id_semantic)
print("Standard: ", kb_id_standard)
print("Hierarchical: ", kb_id_hierarchical)
print("Custom chunking: ", kb_id_custom)

kb_ids = [
    kb_id_standard,
    kb_id_hierarchical,
    kb_id_semantic,
    kb_id_custom
]

kb_names = [
    "Fixed Chunking",
    "Hierarchical Chunking", 
    "Semantic Chunking",
    "Custom Chunking"
]

print(f"설정된 Knowledge Base 수: {len(kb_ids)}")
for name, kb_id in zip(kb_names, kb_ids):
    print(f"- {name}: {kb_id}")

Semantic:  NY2KNVB6RT
Standard:  C3DJFT63RJ
Hierarchical:  OOVXFHUKUA
Custom chunking:  UADADKRAXE
설정된 Knowledge Base 수: 4
- Fixed Chunking: C3DJFT63RJ
- Hierarchical Chunking: OOVXFHUKUA
- Semantic Chunking: NY2KNVB6RT
- Custom Chunking: UADADKRAXE


In [81]:
print("Amazon Bedrock RAG 평가 도구를 사용한 청킹 전략 비교 평가")
print("=" * 80)

# 비교 평가 실행
evaluation_results = run_chunking_strategy_evaluation(
    kb_ids=kb_ids,
    kb_names=kb_names,
    questions=questions,
    ground_truths=ground_truths
)

print("\n평가 실행 완료!")

Amazon Bedrock RAG 평가 도구를 사용한 청킹 전략 비교 평가

=== Fixed Chunking 청킹 전략 평가 시작 ===
질문 1 평가 중...
Fixed Chunking 평가 완료

=== Hierarchical Chunking 청킹 전략 평가 시작 ===
질문 1 평가 중...
Hierarchical Chunking 평가 완료

=== Semantic Chunking 청킹 전략 평가 시작 ===
질문 1 평가 중...
Semantic Chunking 평가 완료

=== Custom Chunking 청킹 전략 평가 시작 ===
질문 1 평가 중...
Custom Chunking 평가 완료

평가 실행 완료!


In [82]:
import re

print("\n" + "=" * 80)
print("평가 결과 요약")
print("=" * 80)

summary_results = {}

for kb_name, results in evaluation_results.items():
    print(f"\n{kb_name} 평가 결과:")
    print("-" * 60)
    
    scores = extract_scores_from_results(results)
    
    avg_recall = 0
    avg_precision = 0
    
    if scores['context_recall']:
        avg_recall = sum(scores['context_recall']) / len(scores['context_recall'])
        print(f"평균 Context Recall: {avg_recall:.4f}")
    
    if scores['context_precision']:
        avg_precision = sum(scores['context_precision']) / len(scores['context_precision'])
        print(f"평균 Context Precision: {avg_precision:.4f}")
    
    summary_results[kb_name] = {
        'avg_recall': avg_recall,
        'avg_precision': avg_precision
    }
    
    # 상세 결과 출력
    for i, result in enumerate(results):
        if 'error' not in result:
            print(f"\n질문 {i+1}: {result['question']}...")
            print(f"생성된 답변: {result['generated_answer']}...")
            print(f"컨텍스트 수: {len(result['contexts'])}")
        else:
            print(f"질문 {i+1} 오류: {result['error']}")

print("\n" + "=" * 80)
print("평가 완료")


평가 결과 요약

Fixed Chunking 평가 결과:
------------------------------------------------------------
평균 Context Recall: 0.1000
평균 Context Precision: 0.4500

질문 1: 2024년 시도별 인구・가구・주택 특성을 종합적으로 비교 분석하시오.(단, 각 시도의 연령구조, 1인가구 비율, 주택 노후도, 외국인 비율을 모두 포함하여 상위 3개 지역과 하위 3개 지역을 각각 제시하고, 이들 간의 특성 차이를 설명하시오)...
생성된 답변: 2024년 시도별 인구・가구・주택 특성을 종합적으로 비교 분석한 결과는 다음과 같습니다.

연령구조 측면에서 상위 3개 지역은 서울 용산구, 경기 의왕시, 대구 남구이며, 하위 3개 지역은 경남 남해군, 경북 영덕군, 인천 동구입니다. 용산구와 의왕시는 상대적으로 젊은 인구 비율이 높은 반면, 남해군과 영덕군은 고령 인구 비율이 높습니다. 1인가구 비율 측면에서 상위 3개 지역은 서울 용산구, 광주 서구, 부산 금정구이며, 하위 3개 지역은 경기 군포시, 경기 의왕시, 대구 남구입니다. 용산구와 서구는 1인가구 비율이 높은 반면, 군포시와 의왕시는 상대적으로 1인가구 비율이 낮습니다. 주택 노후도 측면에서 상위 3개 지역은 서울 용산구, 부산 금정구, 광주 서구이며, 하위 3개 지역은 경기 군포시, 경기 의왕시, 대구 남구입니다. 용산구와 금정구는 상대적으로 주택 노후도가 높은 반면, 군포시와 의왕시는 주택 노후도가 낮습니다. 외국인 비율 측면에서 상위 3개 지역은 서울 양천구, 경기 군포시, 서울 용산구이며, 하위 3개 지역은 경북 영덕군, 경남 남해군, 인천 동구입니다. 양천구와 군포시는 외국인 비율이 높은 반면, 영덕군과 남해군은 외국인 비율이 낮습니다....
컨텍스트 수: 4

Hierarchical Chunking 평가 결과:
----------------------------------------------------

In [83]:
import pandas as pd

# 결과를 DataFrame으로 변환
df_results = pd.DataFrame(summary_results).T
df_results.columns = ['Context Recall', 'Context Precision']

print("\n청킹 전략별 성능 비교:")
print("=" * 50)
print(df_results.round(4))

# 최고 성능 전략 찾기
best_recall = df_results['Context Recall'].idxmax()
best_precision = df_results['Context Precision'].idxmax()

print(f"\n최고 Context Recall: {best_recall} ({df_results.loc[best_recall, 'Context Recall']:.4f})")
print(f"최고 Context Precision: {best_precision} ({df_results.loc[best_precision, 'Context Precision']:.4f})")

# 종합 점수 계산 (Recall과 Precision의 평균)
df_results['Overall Score'] = (df_results['Context Recall'] + df_results['Context Precision']) / 2
best_overall = df_results['Overall Score'].idxmax()

print(f"\n종합 최고 성능: {best_overall} ({df_results.loc[best_overall, 'Overall Score']:.4f})")


청킹 전략별 성능 비교:
                       Context Recall  Context Precision
Fixed Chunking                    0.1               0.45
Hierarchical Chunking             0.8               0.60
Semantic Chunking                 0.2               0.70
Custom Chunking                   0.2               0.60

최고 Context Recall: Hierarchical Chunking (0.8000)
최고 Context Precision: Semantic Chunking (0.7000)

종합 최고 성능: Hierarchical Chunking (0.7000)


In [85]:
print("===============================Knowledge base with fixed chunking==============================\n")
knowledge_base_standard.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
print("===============================Knowledge base with hierarchical chunking==============================\n")
knowledge_base_hierarchical.delete_kb(delete_s3_bucket=False,delete_iam_roles_and_policies=True)
print("===============================Knowledge base with semantic chunking==============================\n")
knowledge_base_semantic.delete_kb(delete_s3_bucket=False,delete_iam_roles_and_policies=True)
print("===============================Knowledge base with custom chunking==============================\n")
knowledge_base_custom.delete_kb(delete_s3_bucket=True,delete_iam_roles_and_policies=True, delete_lambda_function = True)

===============================Knowledge base with fixed chunking==============================

Deleted data source VDO2VETMRQ
======== Knowledge base and all data sources deleted =========
Found bucket standard-kb-9072955
Error deleting bucket standard-kb-9072955: An error occurred (AccessDenied) when calling the ListObjectVersions operation: User: arn:aws:sts::211125368524:assumed-role/AmazonSageMaker-ExecutionRole-20241227T195512/SageMaker is not authorized to perform: s3:ListBucketVersions on resource: "arn:aws:s3:::standard-kb-9072955" because no identity-based policy allows the s3:ListBucketVersions action
======== S3 bucket deletion process completed =========
Found role AmazonBedrockExecutionRoleForKnowledgeBase_9072955-f
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_9072955-f========
 [{'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_9072955-f', 'PolicyArn': 'arn:aws:iam::211125368524:policy/AmazonBedrockFoundationModelPolicyFo